In [ ]:
import sys
sys.path.append('../')
#import matplotlib.pyplot as plt
import numpy as np
import pygmt
#import pandas as pd
#import geopandas as gpd
import pyshtools
#import pygplates
#from remit.earthvim import SeafloorGrid
#from remit.earthvim import SeafloorAgeProfile
from remit.earthvim import PolarityTimescale
#from remit.earthvim import GlobalVIS
from remit.utils.grid import make_dataarray
from gprm.datasets import Seafloor

from remit.data.models import load_lcs, load_ocean_age_model


%load_ext autoreload
%autoreload 2


In [ ]:
ocean = load_ocean_age_model()

pol = PolarityTimescale()
polarity_da = make_dataarray(ocean.lon, ocean.lat, pol.Interpolator(ocean.age))
polarity_da.data[np.isnan(ocean.age.data)] = np.nan


def coeffs2map(coeffs, altitude=300000, lmax=133, lmin=16):
    
    if isinstance(coeffs, str):
        clm, _ = pyshtools.shio.shread(coeffs)
        coeffs = pyshtools.SHMagCoeffs.from_array(clm, r0=6371000.)
        
    coeffs.coeffs[:,:lmin,:lmin] = 0
    x = coeffs.expand(extend=True, a=coeffs.r0+altitude, lmax=lmax)
    return x.rad.to_xarray()


In [ ]:
lmax = 185
cmax = 50

lcs_coeffs = load_lcs()
LCS_rad_0km = coeffs2map(lcs_coeffs, altitude=0, lmax=lmax, lmin=16)
#LCS_rad_0km = coeffs2map('../glm/LCS_mod.cof', altitude=0, lmax=lmax, lmin=16)
LCS_rad_300km = coeffs2map(lcs_coeffs, altitude=300000, lmax=lmax, lmin=16)


In [ ]:
def plot_mag_map(fig, rad, projection, region, cmax):

    radi = pygmt.grdsample(grid=rad, spacing='0.05d')
    pygmt.config(COLOR_FOREGROUND='red', COLOR_BACKGROUND='blue')
    pygmt.makecpt(cmap='polar', series='-{:f}/{:f}'.format(float(cmax), float(cmax)), reverse=True)
    radi.to_netcdf('./grids/_tmp.nc')
    fig.grdimage(region=region, projection=projection, 
                 grid='./grids/_tmp.nc', transparency=10)

    fig.coast(shorelines='0.5p,gray20', resolution='l', area_thresh=5000.,
              region=region, projection=projection, land='gray', transparency=20)
    fig.basemap(frame='f', region=region, projection=projection)
    with pygmt.config(FONT_ANNOT_PRIMARY='6p', FONT_LABEL='8p'):
        fig.colorbar(position='JTR+jTR+o0.5c/0.0c+w2.5c/0.25c+h', frame=['x+lRadial component [nT]'], box='+gwhite+p0.5p')


    
def plot_polarity(fig, polarity_da, projection, region):
    pygmt.makecpt(cmap='hot', series=[-1.5,1.5,1.5], reverse=True, overrule_bg=True)
    polarity_da.to_netcdf('./grids/_tmp.nc')
    fig.grdimage(region=region, projection=projection, 
                 grid = './grids/_tmp.nc',
                 interpolation='n', transparency=40)

    fig.coast(shorelines='0.5p,gray20', resolution='l', area_thresh=5000.,
              region=region, projection=projection, land='gray', transparency=20)
    fig.basemap(frame='f', region=region, projection=projection)
      
    with pygmt.config(FONT_ANNOT_PRIMARY='1p,white', FONT_LABEL='8p', MAP_TICK_LENGTH_PRIMARY='0p', MAP_FRAME_PEN='1p,black'):
        fig.colorbar(position='JBR+jBR+o0.5c/0.0c+w2.5c/0.33c+h', frame=['+n','xa200+lReverse    Normal '], box=['+gwhite+p1p'])


def plot_seafloor_fabric(fig, projection, region):

    pygmt.config(COLOR_FOREGROUND='white', COLOR_BACKGROUND='black')
    pygmt.makecpt(cmap='gray', series='-60/60', reverse=True)
    fig.basemap(frame='afg', region=region, projection=projection)

    fig.plot(data=Seafloor.MagneticPicks(load=False), style='c0.01c', fill='orange', transparency=20, region=region, projection=projection, label='Magnetic Picks')
    fig.plot(data=Seafloor.SeafloorFabric(feature_type='FZ', load=False), pen='0.15p,black', region=region, projection=projection, label='Fracture Zones')
    fig.plot(data=Seafloor.LargeIgneousProvinces(load=False, catalogue='UTIG'), fill='red', transparency=60, region=region, projection=projection, label='Large Igneous Provinces')
    fig.plot(data='./gis/Muller_etal_AREPS_2016_Ridges.gmt', pen='0.3p,dodgerblue', region=region, projection=projection, label='Spreading Ridges')

    fig.coast(shorelines='0.5p,gray20', resolution='l', area_thresh=5000.,
              region=region, projection=projection, land='darkgray', transparency=5)
    fig.basemap(frame='f', region=region, projection=projection)
    
    with pygmt.config(FONT_ANNOT_PRIMARY='8p', FONT_LABEL='8p', MAP_FRAME_PEN='1p,black'):
        fig.legend(position="JBR+jBR+o0.2c/-0.75c+w4.1c", box=['+gwhite+p1p'])


def PlotPanels(fig, projection, region, figsize, cmax):
    with fig.subplot(nrows=2, ncols=3, figsize=figsize, frame="lrtb", autolabel='+gwhite+pblack', margins='0.15i'):
        with fig.set_panel(panel=0, fixedlabel='LCS'):
            plot_mag_map(fig, LCS_rad, projection=projection, region=region, cmax=cmax)
        with fig.set_panel(panel=1, fixedlabel='CRM'):
            plot_mag_map(fig, model_rad_RayLab, projection=projection, region=region, cmax=cmax)
        with fig.set_panel(panel=3, fixedlabel='Pure TRM'):
            plot_mag_map(fig, model_rad_Simple, projection=projection, region=region, cmax=cmax)
        with fig.set_panel(panel=4, fixedlabel='Sloping TRM'):
            plot_mag_map(fig, model_rad_Geek, projection=projection, region=region, cmax=cmax)

        with fig.set_panel(panel=2, fixedlabel='Polarity'):
            plot_polarity(fig, polarity_da, projection, region)
        with fig.set_panel(panel=5, fixedlabel='Seafloor Fabric'):
            plot_seafloor_fabric(fig, projection, region)
            

In [ ]:
projection='N0/4i'
region = 'g'
    
fig = pygmt.Figure()

pygmt.config(FONT_TAG='12p')
with fig.subplot(nrows=2, ncols=2, figsize=(21,11), frame="lrtb", autolabel='+gwhite+pblack', margins='0.02i'):
    with fig.set_panel(panel=0, fixedlabel='a) LCS, 0 km'):
        plot_mag_map(fig, LCS_rad_0km, projection=projection, region=region, cmax=50)
    with fig.set_panel(panel=1, fixedlabel='b) LCS, 300 km'):
        plot_mag_map(fig, LCS_rad_300km, projection=projection, region=region, cmax=10)
    with fig.set_panel(panel=2, fixedlabel='c) Seafloor magnetization polarity'):
        plot_polarity(fig, polarity_da, projection, region)
    with fig.set_panel(panel=3, fixedlabel='d) Magnetic Age Picks and Seafloor Fabric'):
        plot_seafloor_fabric(fig, projection, region)

fig.savefig('./figures/figure1_global_panels_2x2.png')
fig.show(width=1000)
